In [ ]:
import cv2 as cv
import numpy as np


def cv_show_image(image):
    cv.imshow("Image",image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
green_lower_limit = np.array([45,100,100])
green_upper_limit = np.array([75,255,255])

red_lower_limit = np.array([17,100,100])
red_upper_limit = np.array([20,255,255])

blue_lower_limit = np.array([110,50,50])
blue_upper_limit = np.array([130,255,255])

skin_tone_lower = np.array([0, 10, 60], dtype = "uint8") 
skin_tone_upper = np.array([20, 150, 255], dtype = "uint8")

In [ ]:
def nothing(x):
    pass

cv.namedWindow("Wind")
cv.createTrackbar("LH","Wind",0,255,nothing)
cv.createTrackbar("HH","Wind",0,255,nothing)

cap = cv.VideoCapture("volleyball_match.mp4")
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))


fourcc = cv.VideoWriter_fourcc(*'MJPG')
writer = cv.VideoWriter('output5.avi', fourcc, 30, (width, height), isColor=True)
min_radius = []
  
while(1):
    _,frame = cap.read()
    
    if(frame is None):
        print("None Frame")
        break
        
    frame_blur = cv.GaussianBlur(frame,(11,11),0)
    frame_dilate = cv.dilate(frame_blur,None, iterations = 2)
    frame_in_hsv = cv.cvtColor(frame_blur,cv.COLOR_BGR2HSV)
    
    #Add trackbars for Hue Range
    lh = cv.getTrackbarPos("LH","Wind")
    hh = cv.getTrackbarPos("HH","Wind")

    yellow_lower_Limit = np.array([lh,100,100])
    yellow_upper_Limit = np.array([hh,255,255])
    
    mask_first_team = cv.inRange(frame_in_hsv,yellow_lower_Limit,yellow_upper_Limit)
    mask_second_team = cv.inRange(frame_in_hsv,red_lower_limit,red_upper_limit)
    
    cnts,heirarchy = cv.findContours(mask_first_team,cv.RETR_LIST,cv.CHAIN_APPROX_NONE)
    red_cnts,heirarchy = cv.findContours(mask_second_team,cv.RETR_LIST,cv.CHAIN_APPROX_NONE)
    yellow_player_count = 0
    
    if(len(cnts) > 0):
        center = None 
        for c in cnts:
            ((x, y), radius) = cv.minEnclosingCircle(c)
            approx = cv.approxPolyDP(c, .03 * cv.arcLength(c, True), True)
            M = cv.moments(c)
            try:
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            except:
                center = None
            x,y,w,h = cv.boundingRect(c)
            if(h>=1.5*w):
                yellow_player_count += 1
            elif(radius < 3 and len(approx) > 4):
                min_radius.append(radius)
                cv.circle(frame, (int(x), int(y)), int(radius),(0, 0, 255), 20)
                cv.circle(frame, center, 5, (0, 0, 255), 10)
                cv.drawContours(frame,c,-1,(255,0,0),10)
               

    red_player_count = 0
    for c in red_cnts:
        x,y,w,h = cv.boundingRect(c)

        #Detect players
        if(h>=3*w):
            red_player_count += 1

        
    print("Num Yellow Players:",yellow_player_count,"Red Players:",red_player_count)    
    cv.imshow("Wind",frame)
    writer.write(frame)
    key = cv.waitKey(6)
    if(key & 0xff == ord('q')):
        print(key)
        break
    

cap.release()
cv.destroyAllWindows()